# Numerical Integration and Reaction Kinetics

In addition to processing experimental data from the real world, Python can also be used to generate simulations of physical systems that change over time. In this notebook, we will practice performing numerical integration to solve systems of differential equations that describe chemical systems.

To simulate a physical system, we need to describe how that system changes over time as a function of its current state. This description often takes the form of a system of ordinary differential equations (ODEs). Although solving ODEs analytically is sometimes difficult or impossible, their solutions can be approximated by numerically integrating them over time, given some initial conditions. Python provides a collection of powerful general-purpose numeral integration tools that can be used for solving an initial value problem (IVP) of this kind. We will be using the `solve_ivp` function for this purpose. The `solve_ivp` function takes three inputs:

1. An arbitrary function describing the derivative of the variable(s)
2. A time span on which to compute the solution to the IVP
3. The initial conditions at the beginning of the time span

The function returns a bundle of information to us. In particular it gives us the following:

1. An array of times within the range specified in the input
2. The value of the function at every time in the array

Learn more about how `solve_ivp` works here: https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html

## Example 1: Radioactive Decay

You have successfully synthesized a 10 mg sample of yttrium-87 and want to how much will be left after a month. Since $^{87}\text{Y}$ undergoes beta decay to $^{87}\text{Sr}$ with a half-life of about $t_{1/2} \approx 3.4\ \text{days}$, we can describe the amount of $^{87}\text{Y}$ over time with the following initial value problem.

$$ \frac{\text{d}y}{\text{d}t} = - \frac{\ln(2)}{t_{1/2}}y \qquad \qquad y(0) = y_0 $$

Here $y$ is the mass of yttrium-87 that changes over time $t$, while and $y_0 = 10\ \text{mg}$ is the initial amount at $t=0$. Here's how we compute the solution in Python:

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# define constants
t12 = 3.4  # half-life of 3.4 days
y0 = [10] # starting with 10 mg (has to be in a list or array)

# the derivitive of y as a function of t and y
def yprime(t, y):
    return - (np.log(2) / t12) * y

# we want to see how the system changes over one month
t_span = [0, 31]

# compute the solution
sol = solve_ivp(yprime, t_span, y0)

# unpack the pieces we want
t = sol.t    # an array of times
y = sol.y[0] # the value of the function at each time

# plot the results
plt.figure(figsize=(10,3))
plt.plot(t, y)
plt.title("Mass of yttrium-87 over time")
plt.xlabel("time (days)")
plt.ylabel("mass (mg)")
plt.show()

The solution makes sense because if we solve this IVP analytically by normal methods of solving differential equations, we obtain a decaying exponential function. Try modifying $t_{1/2}$ and $y_0$ to see how the output changes. Although an analytical solution is easy to obtain for this system, using Python is much easier for more complex IVPs.

You may have noticed a couple of strange things in the example above. When specifying the initial value `y0 = [10]` it was required to contain it inside a list or array. Additionally, we extracted the solution with `sol.y[0]`. The reason for both is that `solve_ivp` is designed to work for IVPs with any number of variables. Next we will explore an example of a such a multi-variable IVP.

## Example 2: Predator-Prey Dynamics

In the nearby area there are populations of both hawks and rabbits. When there are lots of rabbits, the hawks thrive on an abundance of food, decimating the rabbit population. But as their food source dwindles, the hawk population falls, leading to a resurgence of rabbits as they freely reproduce. We can use the [Lotka-Volterra Model](https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations) to simulate this behavior. If $r$ represents the number of rabbits and $h$ represents the number of hawks, then the population dynamics are described by the following IVP.

\begin{align*}
\frac{\text{d}r}{\text{d}t} &= a r - b rh & r(0) &= r_0 \\
\frac{\text{d}h}{\text{d}t} &= -c h + d rh & h(0) &= h_0 \\
\end{align*}

For this simulation, let $a=8$, $b=2$, $c=3$, and $d=1$. Assume we start with $r_0 = 50$ rabbits and $h_0 = 50$ hawks.

In [ ]:
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# define constants
a = 8
b = 2
c = 3
d = 1

# array of initial conditions [r0, h0]
y0 = [50, 50]

# the derivatives of both r0 and h0 over time
def yprime(t, y):
    r = y[0] # unpack arguments
    h = y[1]
    rprime = a*r - b*r*h # compute derivatives
    hprime = -c*h + d*r*h
    return [rprime, hprime] # pack them up again

# specify time span of solution
t_span = [0, 20]

# compute the solution
sol = solve_ivp(yprime, t_span, y0)

# unpack the pieces we want
t = sol.t    # an array of times
r = sol.y[0] # unpack both variables
h = sol.y[1]

# plot the results
plt.figure(figsize=(10,3))
plt.plot(t, r)
plt.plot(t, h)
plt.title("Lotka-Volterra Model")
plt.xlabel("time (years)")
plt.ylabel("population (individuals)")
plt.legend(["rabbits", "hawks"])
plt.show()


As expected, the rabbit and hawk populations oscillate over time.

**Python Question 1**

You can now apply these concepts to simulate a chemical reaction with first-order kinetics. Consider the following reversible association/dissociation reaction. This could represent an acid-base or solubility process, for example.

$$ \text{A} + \text{B} \quad {}_{\xleftarrow[k_2]{}}^{ \xrightarrow{k_1}} \quad \text{AB} \\[0.5em] $$

Assuming a first order kinetics mechanism, the system is described by the following IVP (make sure you understand how this was derived).

$$ \begin{align*}
\frac{\text{d}[\text{A}]}{\text{d}t} &= - k_1 [\text{A}][B] + k_2[\text{AB}] & \left [\text{A}] \right |_{t=0} &= [\text{A}]_0 \\
\frac{\text{d}[\text{B}]}{\text{d}t} &= - k_1 [\text{A}][\text{B}] + k_2[\text{AB}] & \left [\text{B}] \right |_{t=0} &= [\text{B}]_0 \\
\frac{\text{d}[\text{AB}]}{\text{d}t} &= k_1 [\text{A}][\text{B}] - k_2[\text{AB}] & \left [\text{AB}] \right |_{t=0} &= [\text{AB}]_0
\end{align*} $$

Assume the initial conditions $[\text{A}]_0 = 0.1\ \text{M}$, $[\text{B}]_0 = 0.2\ \text{M}$, and $[\text{AB}]_0 = 0\ \text{M}$. Let the rate constants be $k_1 = 0.5 \ \text{M}^{-1}\text{s}^{-1}$ and $k_2 = 0.01 \ \text{s}^{-1}$. Complete the code below to simulate the reaction over the course of 120 seconds.

In [ ]:
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# define constants
k1 = ???
k2 = ???

# define initial conditions [A0, B0, AB0]
y0 = [???, ???, ???]

# the derivatives of all chemical species over time
def yprime(t, y):
    A, B, AB = y[0], y[1], y[2] # unpack arguments

    Aprime = ??? # compute derivatives
    Bprime = ???
    ABprime = ???
    
    return [Aprime, Bprime, ABprime] # pack them up again

# specify time span of solution
t_span = [0, 20]

# compute the solution
sol = solve_ivp(yprime, t_span, y0)

# unpack the pieces we want
t = sol.t    # an array of times
A = sol.y[0] # unpack both variables
B = sol.y[1]
C = sol.y[2]

# plot the results
plt.figure(figsize=(10,3))
plt.plot(t, A)
plt.plot(t, B)
plt.plot(t, C)
plt.title("First Order Kinetics")
plt.xlabel("time (s)")
plt.ylabel("concentration (M)")
plt.legend(["[A]", "[B]", "[C]"])
plt.show()
